# Evolver Loop 1 Analysis

Analyzing the current state and planning next experiments.

In [ ]:
import pandas as pd
import numpy as np
import os
import json

# Load session state
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

print("Current Experiments:")
for exp in state['experiments']:
    print(f"  {exp['name']}: CV={exp['cv_score']:.6f}, LB={exp.get('lb_score', 'N/A')}")

print(f"\nTarget: 68.919154")
print(f"Best CV: {min(exp['cv_score'] for exp in state['experiments']):.6f}")
print(f"Gap: {min(exp['cv_score'] for exp in state['experiments']) - 68.919154:.6f}")

In [ ]:
# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

# Calculate bounding box for a single tree at different angles
def get_bbox_side(angle_deg):
    angle_rad = np.radians(angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    
    # Rotate vertices
    x_rot = [cos_a * x - sin_a * y for x, y in zip(TX, TY)]
    y_rot = [sin_a * x + cos_a * y for x, y in zip(TX, TY)]
    
    # Bounding box
    side = max(max(x_rot) - min(x_rot), max(y_rot) - min(y_rot))
    return side

# Find optimal angle for N=1
angles = np.arange(0, 360, 0.001)
sides = [get_bbox_side(a) for a in angles]
best_idx = np.argmin(sides)
print(f"Optimal angle for N=1: {angles[best_idx]:.3f} degrees")
print(f"Optimal side length: {sides[best_idx]:.6f}")
print(f"Score contribution: {sides[best_idx]**2:.6f}")

In [ ]:
# Check current N=1 configuration
preopt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv'
df = pd.read_csv(preopt_path)

# Parse N=1
n1_row = df[df['id'] == '001_0'].iloc[0]
x = float(str(n1_row['x']).replace('s', ''))
y = float(str(n1_row['y']).replace('s', ''))
deg = float(str(n1_row['deg']).replace('s', ''))

print(f"Current N=1 configuration:")
print(f"  x={x}, y={y}, deg={deg}")
print(f"  Side length: {get_bbox_side(deg):.6f}")
print(f"  Score contribution: {get_bbox_side(deg)**2:.6f}")

In [ ]:
# Check if 45 degrees is optimal
print("\nComparing angles around 45 degrees:")
for angle in [44.5, 44.75, 45.0, 45.25, 45.5]:
    side = get_bbox_side(angle)
    print(f"  {angle:.2f}°: side={side:.6f}, score={side**2:.6f}")

In [ ]:
# Analyze per-N scores from the baseline
from shapely.geometry import Polygon
from shapely import affinity

TREE_VERTICES = list(zip(TX, TY))

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        
        initial_polygon = Polygon(TREE_VERTICES)
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def compute_bounding_side(trees):
    all_points = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_points.append(coords)
    all_points = np.vstack(all_points)
    side = max(all_points.max(axis=0) - all_points.min(axis=0))
    return side

print("Per-N analysis (top 20 contributors):")
per_n_scores = []
for n in range(1, 201):
    trees = load_trees_for_n(df, n)
    side = compute_bounding_side(trees)
    contribution = side**2 / n
    per_n_scores.append({'n': n, 'side': side, 'contribution': contribution, 'efficiency': n / (side**2)})

per_n_df = pd.DataFrame(per_n_scores)
per_n_df = per_n_df.sort_values('contribution', ascending=False)
print(per_n_df.head(20).to_string())

In [ ]:
# Summary
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Target score: 68.919154")
print(f"Current best: 70.676102")
print(f"Gap: 1.756948 (2.55%)")
print("\nKey insights from previous experiments:")
print("1. Pre-optimized santa-2025.csv is at a VERY TIGHT local optimum")
print("2. Standard optimizers (SA, backward propagation) found NO improvements")
print("3. Small N values (1-10) have highest score contribution")
print("4. N=1 is already at optimal 45-degree angle")
print("\nNext steps to try:")
print("1. Fractional translation (micro-adjustments at 0.001-0.00001 scale)")
print("2. Lattice-based packing for large N (72, 100, 144, 196, 200)")
print("3. Much longer optimization runs (hours, not minutes)")
print("4. Population-based optimization with perturbation")